# Decorators in Python

This notebook offers a quick glimpse at decorators in Python. For a more complete tour about decorators, please consult the following websites/books this notebook takes inspiration from:
* [A guide to Python's function decorators](https://www.thecodeship.com/patterns/guide-to-python-function-decorators/)
* [PythonWiki](https://wiki.python.org/moin/PythonDecorators)
* [PEP 318 -- Decorators for Functions and Methods](https://www.python.org/dev/peps/pep-0318/)
* [Primer on Python Decorators](https://realpython.com/blog/python/primer-on-python-decorators/)
* [Professional Python -- Chapter 1](https://www.amazon.fr/Professional-Python-Luke-Sneeringer/dp/1119070856)

## Functions of functions

Python's functions can take any `object` as an argument, including other functions. Consider the following example:

In [1]:
def print_name(func):
    print("Function name:", func.__name__)

def foo(a, b):
    return a + b


print_name(foo)

Function name: foo


Note that the function `foo` did not need to be executed (nor given arguments in `print_name`). The function `print_name` acts on the **function**, not on its result.

## Returning functions

A more interesting feature of Python's functions is that they can return other functions; even functions defined **inside** the current scope.

In [2]:
def create_answering_machine(message):
    def answer():
        print(message)

    return answer


busy = create_answering_machine("I'm busy.")
absent = create_answering_machine("I'm not here.")

busy()
absent()

I'm busy.
I'm not here.


In the above example, `create_answering_machine` takes a string as an argument and returns a *function* printing the input string when called, providing a way of easily creating customized functions. Note that `busy` and `absent` must be followed by parenthesis in order to print anything, since they are functions. 

## Why not both?

Now let us see what can happen when we define a function taking a function as an argument **and** returning a function.

In [3]:
def say_when_executed(func):
    def new_func():
        print("Executing ", func.__name__, "...", sep='')
        func()
        print("Done!")

    return new_func

def foo():
    print("Dummy function.")


foo()
foo = say_when_executed(foo)
foo()

Dummy function.
Executing foo...
Dummy function.
Done!


The function `say_when_executed` takes a function as an input and return the same function, except it now prints some the function's name before and after its execution. This seems to be an interesting logging tool. We could improve the previous example to extend it to functions with arguments, using the `*args` and `**kwargs` notation:

In [4]:
def improved_logs(func):
    def new_func(*args, **kwargs):
        print("Executing ", func.__name__, "...", sep='')
        func(*args, **kwargs)
        print("Done!")

    return new_func

def add(a, b):
    print(a + b)


add(1, 2)
add = improved_logs(add)
add(1, 2)

3
Executing add...
3
Done!


## A more readable way: decorators

The last example offers a great way to easily generate logs when executing specific functions. However, this can be make easier and more readable thanks to decorators. Decorators are functions that take a function as an argument and return another function (more precisely, they can take any `callable` as an argument - including classes - but this is out of the scope of this notebook). A decorator is called right before a function definition, using the `@` symbol. The previous example can be rewritten as follows:

In [5]:
def improved_logs(func):
    def new_func(*args, **kwargs):
        print("Executing ", func.__name__, "...", sep='')
        func(*args, **kwargs)
        print("Done!")

    return new_func

@improved_logs
def add(a, b):
    print(a + b)


add(1, 2)

Executing add...
3
Done!


The `improved_logs` notation is exactly the same as `add = improved_logs(add)`.

We could also use a decorator to monitor a function execution time.

In [6]:
import time

def execution_time(func):
    def new_func(*args, **kwargs):
        t = time.time()
        func(*args, **kwargs)
        print("Function", func.__name__, "executed in",
              time.time() - t, "seconds.")

    return new_func

@execution_time
def just_wait(sec):
    time.sleep(sec)


just_wait(2)

Function just_wait executed in 2.0005388259887695 seconds.


## Useful decorators

### Logging

Often when monitoring or debugging a script execution, it can be useful to know the order in which functions are called, or even if they are called at all. Instead of writing a bunch of `print` statement at the beggining of every function, decorators offer a more elegant way to alter functions of interest.

In [7]:
def log_it(func):
    def new_func(*args, **kwargs):
        print("[LOG] Entered", func.__name__)
        func(*args, **kwargs)
        print("[LOG] Exited", func.__name__)

    return new_func

@log_it
def interesting_function():
    print("I am an interesting function.")

@log_it
def very_interesting_function():
    print("I am even more interesting.")
    interesting_function()

def not_interesting_function():
    print("Nobody cares.")


interesting_function()
not_interesting_function()
very_interesting_function()

[LOG] Entered interesting_function
I am an interesting function.
[LOG] Exited interesting_function
Nobody cares.
[LOG] Entered very_interesting_function
I am even more interesting.
[LOG] Entered interesting_function
I am an interesting function.
[LOG] Exited interesting_function
[LOG] Exited very_interesting_function


This is a very simple example about logging. For better practices and in-depth logging tools explanations, one should check the [Python's Logging HOWTO](https://docs.python.org/3/howto/logging.html).

### Monitoring execution time

Another application of decorators we mentioned earlier concerns functions execution times.

In [8]:
def time_it(func):
    def new_func(*args, **kwargs):
        t = time.time()
        func(*args, **kwargs)
        exec_time = time.time() - t
        print(func.__name__, "'s execution time: ",
              "{0:.2f}".format(exec_time), " second(s).",
              sep='')

    return new_func

@log_it
@time_it
def a_long_func():
    time.sleep(5)


a_long_func()

[LOG] Entered new_func
a_long_func's execution time: 5.00 second(s).
[LOG] Exited new_func


Here we combined two decorators: `log_it` and `time_it`. They have no influence on the function itself (they are just 'printers') so their order is not critical, expect for the printing order. Note however that if you are chaining operators that modify the core function, they are executed in order from bottom to top. In other word, the code
```
@decorator_a
@decorator_b
def foo()
    return 0
```
is equivalent to
```
def foo()
    return 0

foo = decorator_a(decorator_b(foo))
```

### Caching results

The final application examples we will see concerns the possibility of caching some function results, in order to avoid useless recomputations. For the sake of clarity, we will not be implementing this functionality here, but rather using the `lru_cache` decorator from the `functools` module.

In [9]:
from functools import lru_cache

@lru_cache(maxsize=2000)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)


fib.cache_clear()
st = time.time()
fib(600)
end1 = time.time()
fib(600)
end2 = time.time()

print("First time: {0:.2e} sec".format(end1 - st))
print("Second time: {0:.2e} sec".format(end2 - end1))

First time: 5.02e-04 sec
Second time: 0.00e+00 sec


In this case, `lru_cache` is called using an argument. This is another extension of decorators that we will not cover here. For further details and explanations, feel free to read tutorials mentioned earlier and references therein.

## Try it!

Decorators can be very convenient tools for extending the functionalities of your code, monitoring a script behavior and keeping a clean code. Moreover, thanks to the Python's community being very active, they are tons of useful decorators for you to try out, so you probably won't have to even implement one anytime soon. Don't hesitate to try them out and share those that you find particularly useful.